# SlideRule Interactive Feature Find

Instructors: [Tyler Sutterley](mailto:tsutterl@uw.edu), Nathan Kurtz and Ben Smith

[Documentation for using SlideRule](https://slideruleearth.io/rtd) is available from the [project website](https://slideruleearth.io) 

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
from sliderule import icesat2, ipysliderule, sliderule, io
import ipywidgets as widgets
import warnings
# autoreload
%load_ext autoreload
%autoreload 2
# turn off warnings for demo
warnings.filterwarnings('ignore')

#### Paste `Show Code` contents from [Web Demo](https://demo.slideruleearth.io/)

### Review GeoDataFrame output
Can inspect the columns, number of returns and returns at the top of the GeoDataFrame.

See the [ICESat-2 documentation](https://slideruleearth.io/web/rtd/user_guide/ICESat-2.html#segmented-photon-data-atl03) for descriptions of each column

In [ ]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

### Interactive Mapping with Leaflet

<table>
  <tbody>
    <tr>
      <th align='center' max-width="30%"><a href="https://epsg.io/3857">Global (Web Mercator, EPSG:3857)</a></th>
      <th align='center' max-width="30%"><a href="https://epsg.io/5936">North (Alaska Polar Stereographic, EPSG:5936)</a></th>
      <th align='center' max-width="30%"><a href="https://epsg.io/3031">South (Antarctic Polar Stereographic, EPSG:3031)</a></th>
    </tr>
    <tr>
      <td align='left' valign='top' max-width="30%">
        <ul style="line-height: 1.5em">
            <li><a href="https://apps.nationalmap.gov/3depdem/">USGS 3DEP Hillshade</a></li>
            <li><a href="https://asterweb.jpl.nasa.gov/gdem.asp">ASTER GDEM Hillshade</a></li>
            <li><a href="https://www.arcgis.com/home/item.html?id=10df2279f9684e4a9f6a7f08febac2a9">ESRI Imagery</a></li>
            <li><a href="http://glims.colorado.edu/glacierdata/">Global Land Ice Measurements from Space (GLIMS)</a></li>
            <li><a href="https://www.glims.org/RGI/">Randolph Glacier Inventory (RGI)</a></li>
        </ul>
       </td>
       <td align='left' valign='top' max-width="30%">
        <ul style="line-height: 1.5em">
            <li><a href="http://goto.arcgisonline.com/maps/Arctic_Imagery">ESRI Imagery</a></li>
            <li><a href="https://www.pgc.umn.edu/data/arcticdem">ArcticDEM</a></li>
        </ul>
       </td>
       <td align='left' valign='top' max-width="30%">
        <ul style="line-height: 1.5em">
            <li><a href="https://lima.usgs.gov/">Landsat Image Mosaic of Antarctica (LIMA)</a></li>
            <li><a href="https://nsidc.org/data/nsidc-0280">MODIS Mosaic of Antarctica (MOA)</a></li>
            <li><a href="https://nsidc.org/data/NSIDC-0103">Radarsat Antarctic Mapping Project (RAMP)</a></li>
            <li><a href="https://www.pgc.umn.edu/data/rema">Reference Elevation Model of Antarctica (REMA)</a></li>
        </ul>
      </td>
    </tr>
  </tbody>
</table>

In [ ]:
# display widgets for setting SlideRule parameters
SRwidgets = ipysliderule.widgets()
SRwidgets.set_atl03_defaults()
widgets.VBox([
    SRwidgets.projection,
    SRwidgets.layers,
    SRwidgets.raster_functions
])

In [ ]:
# center map on polygon centroid
lon, lat = io.from_region(parms['poly'])
cx, cy = io.centroid(lon, lat)
# create ipyleaflet map in specified projection
m = ipysliderule.leaflet(SRwidgets.projection.value,
    center=(cy, cx), prefer_canvas=True)
# install click handler callback
m.add_selected_callback(SRwidgets.atl03_click_handler)
# add polygon from web demo code
m.add_region([parms['poly']])
# show map
m.map

In [ ]:
# add any selected base layers
m.add_layer(
    layers=SRwidgets.layers.value,
    rendering_rule=SRwidgets.rendering_rule
)

#### Add GeoDataFrame to map

For stability of the leaflet map, SlideRule will as a default limit the plot to have up to 10000 points from the GeoDataFrame

GeoDataFrames can be plotted in any available [matplotlib colormap](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse,
])

In [ ]:
%matplotlib inline
# ATL03 fields for hover tooltip
fields = m.default_atl03_fields()
m.GeoData(gdf, column_name=SRwidgets.variable.value, cmap=SRwidgets.colormap,
    max_plot_points=10000, tooltip=True, colorbar=True, fields=fields)

#### Visualize photons as an along-track plot

In [ ]:
widgets.VBox([
    SRwidgets.plot_classification,
    SRwidgets.rgt,
    SRwidgets.ground_track,
    SRwidgets.cycle,
])

In [ ]:
%matplotlib widget
SRwidgets.plot(atl03=gdf, kind='scatter', title='Photon Cloud',
    cmap=SRwidgets.colormap, legend=True, legend_frameon=True,
    classification=SRwidgets.plot_classification.value, 
    segments=False)